# LIBERO Complete Walkthrough: Gym & Benchmark

This notebook provides a **comprehensive guide** to working with LIBERO in PhysicalAI:

**Part 1: Understanding LIBERO**
- What is LIBERO and its task suites
- Installation and setup

**Part 2: LiberoGym (Low-Level)**
- Creating environments
- Observation and action spaces
- Running single rollouts

**Part 3: Policy Creation**
- First-party ACT policy
- Third-party LeRobot policies (Diffusion, VQ-BeT)
- Loading from datasets

**Part 4: Evaluation Tools**
- `rollout()` - Single episode evaluation
- `Rollout` metric - TorchMetrics for Lightning
- `evaluate_policy()` - Multi-episode evaluation

**Part 5: LiberoBenchmark (High-Level)**
- Benchmark class for standardized evaluation
- Multi-task evaluation
- Results export (JSON, CSV)
- Video recording

## 1. Setup & Imports

In [ ]:
# Core imports
import matplotlib.pyplot as plt
import numpy as np
import torch
from physicalai.gyms.libero import LiberoGym

# PhysicalAI imports
from physicalai.data import Feature, FeatureType, NormalizationParameters
from physicalai.devices import get_available_device
from physicalai.policies import ACT, ACTModel

# Check device (supports CUDA, XPU, and CPU)
device = get_available_device()
print(f"Using device: {device}")

## 2. LIBERO Benchmark Overview

| Suite | Tasks | Max Steps | Focus |
|-------|-------|-----------|-------|
| `libero_spatial` | 10 | 280 | Spatial reasoning (same objects, different positions) |
| `libero_object` | 10 | 280 | Object generalization (different objects, same actions) |
| `libero_goal` | 10 | 300 | Goal-conditioned tasks (same scene, different goals) |
| `libero_10` | 10 | 520 | Mixed difficulty benchmark |
| `libero_90` | 90 | 400 | Large-scale comprehensive benchmark |

In [ ]:
try:
    from libero.libero import benchmark
except ModuleNotFoundError:
    msg = "LIBERO is not installed. Install it with:\n  pip install hf-libero\nOr with uv:\n  uv pip install hf-libero"
    raise ImportError(msg) from None

# List available tasks
for suite_name in ["libero_spatial", "libero_object"]:
    suite = benchmark.get_benchmark_dict()[suite_name]()
    tasks = suite.get_task_names()
    print(f"\n{suite_name} ({len(tasks)} tasks):")
    for i, task in enumerate(tasks[:3]):
        print(f"  [{i}] {task}")
    if len(tasks) > 3:
        print(f"  ... and {len(tasks) - 3} more")

## 3. Create LiberoGym Environment

In [ ]:
# Create a LiberoGym instance
gym = LiberoGym(
    task_suite="libero_spatial",
    task_id=0,
    observation_height=256,
    observation_width=256,
    obs_type="pixels_agent_pos",  # Images + proprioception
    control_mode="relative",  # Delta actions
)

print(f"Task: {gym.task_name}")
print(f"Max episode steps: {gym.max_episode_steps}")
print(f"Action space: {gym.action_space.shape}")

In [ ]:
# Reset and inspect observation
obs, info = gym.reset(seed=42)

print(f"Observation type: {type(obs).__name__}")
print(f"\nImages: {list(obs.images.keys())}")
for name, img in obs.images.items():
    print(f"  {name}: {img.shape}")

print(f"\nState: {obs.state.shape}")
print("  Format: [eef_pos(3), axis_angle(3), gripper(2)]")
print(f"  Values: {obs.state.squeeze().numpy()}")

In [ ]:
# Visualize camera views
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

img1 = obs.images["image"].squeeze(0).permute(1, 2, 0).numpy()
img2 = obs.images["image2"].squeeze(0).permute(1, 2, 0).numpy()

axes[0].imshow(img1)
axes[0].set_title("Front Camera (agentview)")
axes[0].axis("off")

axes[1].imshow(img2)
axes[1].set_title("Eye-in-Hand Camera")
axes[1].axis("off")

plt.suptitle(f"Task: {gym.task_name[:50]}...")
plt.tight_layout()
plt.show()

## 4. Define Policy Features (Manual Approach)

PhysicalAI uses `Feature` dataclasses to define input/output shapes. This is the **manual approach** - you can also use `ACT.from_dataset()` for automatic feature extraction (see Section 5).

In [ ]:
# Input features matching LiberoGym output (with normalization for ACT)
input_features = {
    "image": Feature(
        ftype=FeatureType.VISUAL,
        shape=(3, 256, 256),
        normalization_data=NormalizationParameters(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ),
    "image2": Feature(
        ftype=FeatureType.VISUAL,
        shape=(3, 256, 256),
        normalization_data=NormalizationParameters(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ),
    "state": Feature(
        ftype=FeatureType.STATE,
        shape=(8,),
        normalization_data=NormalizationParameters(mean=[0.0] * 8, std=[1.0] * 8),
    ),
}

# Output features (7-dim action)
output_features = {
    "action": Feature(
        ftype=FeatureType.ACTION,
        shape=(7,),
        normalization_data=NormalizationParameters(mean=[0.0] * 7, std=[1.0] * 7),
    ),
}

print("Input features:")
for name, feat in input_features.items():
    print(f"  {name}: {feat.ftype.name} {feat.shape}")
print(f"\nOutput: action {output_features['action'].shape}")

## 5. Create First-Party ACT Policy

Two approaches to create an ACT policy:
1. **Manual**: Define features yourself (shown above)
2. **From Dataset**: Use `ACT.from_dataset()` for automatic feature extraction

In [ ]:
# Option 1: Manual feature definition (from Section 4)
act_model = ACTModel(
    input_features=input_features,
    output_features=output_features,
    chunk_size=100,
    dim_model=256,
    n_encoder_layers=2,
    n_decoder_layers=1,
)
act_policy = ACT(model=act_model)
act_policy.to(device)
act_policy.eval()

print(f"✅ First-party ACT policy created on {device}")
print(f"   Parameters: {sum(p.numel() for p in act_policy.parameters()):,}")

### Option 2: Using `ACT.from_dataset()`

This approach automatically extracts features from a dataset:

In [ ]:
# Option 2: Create ACT from a dataset (automatic feature extraction)
from physicalai.data.lerobot import LeRobotDataModule

# Create datamodule from a compatible LeRobot dataset
# For LIBERO, use "lerobot/libero_spatial" once migrated to v3.0 format
datamodule = LeRobotDataModule(
    repo_id="lerobot/aloha_sim_transfer_cube_human",  # Example with ALOHA (2 arms, 4 cameras)
    train_batch_size=32,
    data_format="physicalai",  # Use PhysicalAI's Observation format
)

# Create ACT policy directly from dataset (features extracted automatically)
act_policy_from_dataset = ACT.from_dataset(
    dataset=datamodule.train_dataset,
    chunk_size=100,
    dim_model=256,
    n_encoder_layers=2,
    n_decoder_layers=1,
)
act_policy_from_dataset.to(device)
act_policy_from_dataset.eval()

print(f"✅ ACT.from_dataset() created policy on {device}")
print(f"   Parameters: {sum(p.numel() for p in act_policy_from_dataset.parameters()):,}")
print("\n📊 Features extracted automatically from dataset:")
print(f"   Observation keys: {list(datamodule.train_dataset.observation_features.keys())}")
print(f"   Action keys: {list(datamodule.train_dataset.action_features.keys())}")

## Part 3: Evaluation Tools

The `physicalai.eval.rollout` module provides production-ready evaluation with:
- Success tracking from environment info
- FPS monitoring  
- Frame collection for visualization
- Chunked action handling (ACT policies)
- Video recording support
- Distributed evaluation support (TorchMetrics)

In [ ]:
# Import the production rollout function
from physicalai.eval.rollout import rollout

# Run single episode with frames for visualization
print("Running policy evaluation with production rollout()...")
print("=" * 60)

result = rollout(gym, act_policy, max_steps=30, seed=42, return_frames=True)

print(f"Episode Length: {result['episode_length']} steps")
print(f"Sum Reward: {result['sum_reward'].item():.4f}")
print(f"Success: {result['success'].item() if 'success' in result else 'N/A'}")
print(f"FPS: {result['fps']:.1f}")
print(f"Frames collected: {len(result['frames'])}")

# Store for visualization
act_result = {
    "frames": result["frames"],
    "actions": result["action"].squeeze(1).cpu().numpy(),
    "sum_reward": result["sum_reward"].item(),
    "steps": result["episode_length"],
    "fps": result["fps"],
    "success": result["success"].item() if "success" in result else False,
}

## 7. Visualize Rollout

Extract frames from observations returned by rollout:

In [ ]:
# Visualize ACT rollout trajectory
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
indices = np.linspace(0, len(act_result["frames"]) - 1, 5, dtype=int)
for i, idx in enumerate(indices):
    axes[i].imshow(act_result["frames"][idx])
    axes[i].set_title(f"Step {idx}")
    axes[i].axis("off")

plt.suptitle("ACT Policy Rollout", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize actions
actions = act_result["actions"]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(actions[:, :3])
axes[0].legend(["Δx", "Δy", "Δz"])
axes[0].set_title("Position Actions")
axes[0].set_xlabel("Step")
axes[0].grid(alpha=0.3)

axes[1].plot(actions[:, 3:6])
axes[1].plot(actions[:, 6], "k--", linewidth=2, label="gripper")
axes[1].legend(["Δroll", "Δpitch", "Δyaw", "gripper"])
axes[1].set_title("Rotation + Gripper Actions")
axes[1].set_xlabel("Step")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Third-Party Integration (LeRobot Policies)

PhysicalAI also supports LeRobot policies (Diffusion, ACT, VQ-BeT, etc.) via `LeRobotPolicy` wrapper:

In [ ]:
# LeRobot integration uses different feature format
from lerobot.configs.types import FeatureType as LRFeatureType
from lerobot.configs.types import PolicyFeature

from physicalai.policies.lerobot import LeRobotPolicy

# LeRobot-style features (note the different naming convention)
lr_input_features = {
    "observation.images.image": PolicyFeature(type=LRFeatureType.VISUAL, shape=(3, 256, 256)),
    "observation.images.image2": PolicyFeature(type=LRFeatureType.VISUAL, shape=(3, 256, 256)),
    "observation.state": PolicyFeature(type=LRFeatureType.STATE, shape=(8,)),
}
lr_output_features = {
    "action": PolicyFeature(type=LRFeatureType.ACTION, shape=(7,)),
}

# Create LeRobot Diffusion policy
# Pass config kwargs directly as **kwargs
diffusion_policy = LeRobotPolicy(
    policy_name="diffusion",
    input_features=lr_input_features,
    output_features=lr_output_features,
    crop_shape=None,  # Pass directly as kwarg
)
diffusion_policy.to(device)
diffusion_policy.eval()

print(f"✓ LeRobot Diffusion policy created: {type(diffusion_policy._lerobot_policy).__name__}")

In [ ]:
# Compare first-party ACT vs LeRobot Diffusion using production rollout
print("Comparing policies with production rollout()...")

print("\n1. First-party ACT:")
act_result = rollout(gym, act_policy, max_steps=20, seed=42, return_frames=True)
print(f"   Steps: {act_result['episode_length']}, Success: {act_result.get('success', 'N/A')}, FPS: {act_result['fps']:.1f}")

print("\n2. LeRobot Diffusion:")
diffusion_result = rollout(gym, diffusion_policy, max_steps=20, seed=42, return_frames=True)
print(f"   Steps: {diffusion_result['episode_length']}, Success: {diffusion_result.get('success', 'N/A')}, FPS: {diffusion_result['fps']:.1f}")

In [ ]:
# Side-by-side comparison using frames from production rollout
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# ACT trajectory (frames from rollout)
act_frames = act_result["frames"]
indices = np.linspace(0, len(act_frames) - 1, 5, dtype=int)
for i, idx in enumerate(indices):
    axes[0, i].imshow(act_frames[idx])
    axes[0, i].set_title(f"Step {idx}")
    axes[0, i].axis("off")
axes[0, 0].set_ylabel("ACT\n(first-party)", fontsize=11)

# Diffusion trajectory (frames from rollout)
diff_frames = diffusion_result["frames"]
indices = np.linspace(0, len(diff_frames) - 1, 5, dtype=int)
for i, idx in enumerate(indices):
    axes[1, i].imshow(diff_frames[idx])
    axes[1, i].set_title(f"Step {idx}")
    axes[1, i].axis("off")
axes[1, 0].set_ylabel("Diffusion\n(LeRobot)", fontsize=11)

plt.suptitle("Policy Comparison: First-Party vs Third-Party (using rollout())", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Demonstrate multi-episode evaluation with Rollout metric
from physicalai.eval.rollout import Rollout, evaluate_policy

print("=" * 60)
print("Multi-Episode Evaluation with Rollout Metric")
print("=" * 60)

# Option 1: Using evaluate_policy function
print("\n1. evaluate_policy() - 3 episodes:")
eval_result = evaluate_policy(gym, act_policy, n_episodes=3, start_seed=0, max_steps=30)
print(f"   Avg Sum Reward: {eval_result['aggregated']['avg_sum_reward']:.4f}")
print(f"   Avg Episode Length: {eval_result['aggregated']['avg_episode_length']:.1f}")
print(f"   Avg FPS: {eval_result['aggregated']['avg_fps']:.1f}")
if "pc_success" in eval_result["aggregated"]:
    print(f"   Success Rate: {eval_result['aggregated']['pc_success']:.1f}%")

# Option 2: Using Rollout metric (TorchMetrics-compatible)
print("\n2. Rollout Metric (TorchMetrics-compatible):")
rollout_metric = Rollout(max_steps=30)
rollout_metric.to(device)

# Run 3 episodes
for ep in range(3):
    rollout_metric.update(gym, act_policy, seed=ep)

# Compute aggregated metrics
metrics = rollout_metric.compute()
print(f"   Avg Sum Reward: {metrics['avg_sum_reward']:.4f}")
print(f"   Success Rate: {metrics['pc_success']:.1f}%")
print(f"   Episodes Evaluated: {metrics['n_episodes']}")

# Get per-episode breakdown
per_ep = rollout_metric.get_per_episode_data()
print("\n   Per-Episode Results:")
for i, ep_data in enumerate(per_ep):
    success_str = "✓" if ep_data.get("success", False) else "✗"
    print(f"   Episode {i + 1}: reward={ep_data['sum_reward']:.4f}, success={success_str}")

# Cleanup
gym.close()
print("\n✅ Environment closed.")

## Part 5: LiberoBenchmark (High-Level Evaluation)

The `LiberoBenchmark` class provides **standardized evaluation** across entire task suites:

| Feature | LiberoGym | LiberoBenchmark |
|---------|-----------|-----------------|
| **Level** | Low-level | High-level |
| **Scope** | Single task | Multiple tasks |
| **Use Case** | Development, debugging | Paper results, comparison |
| **Video Recording** | Manual | Built-in |
| **Results Export** | Manual | JSON, CSV |

In [ ]:
# LiberoBenchmark automatically creates gyms for all tasks in a suite
from physicalai.benchmark import LiberoBenchmark

# Create benchmark for libero_spatial (10 tasks)
# Limit to first 2 tasks for demo speed
benchmark = LiberoBenchmark(
    task_suite="libero_spatial",
    task_ids=[0, 1],  # Evaluate only tasks 0 and 1 (use None for all)
    num_episodes=2,   # 2 episodes per task
    max_steps=30,     # Short episodes for demo
    seed=42,
)

print(f"Benchmark: {benchmark}")
print(f"Tasks: {len(benchmark.gyms)}")
print(f"Episodes per task: {benchmark.num_episodes}")
print(f"Total rollouts: {len(benchmark.gyms) * benchmark.num_episodes}")

### 5.1 Single Policy Evaluation

In [ ]:
# Recreate the ACT policy for benchmark (need to match new gym specs)
# First, get observation specs from benchmark gyms
sample_gym = benchmark.gyms[0]
sample_obs, _ = sample_gym.reset(seed=0)

# Create features matching the benchmark gym
benchmark_input_features = {
    "image": Feature(
        ftype=FeatureType.VISUAL,
        shape=(3, 256, 256),
        normalization_data=NormalizationParameters(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ),
    "image2": Feature(
        ftype=FeatureType.VISUAL,
        shape=(3, 256, 256),
        normalization_data=NormalizationParameters(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ),
    "state": Feature(
        ftype=FeatureType.STATE,
        shape=(8,),
        normalization_data=NormalizationParameters(mean=[0.0] * 8, std=[1.0] * 8),
    ),
}
benchmark_output_features = {
    "action": Feature(
        ftype=FeatureType.ACTION,
        shape=(7,),
        normalization_data=NormalizationParameters(mean=[0.0] * 7, std=[1.0] * 7),
    ),
}

# Create policy for benchmarking
# Note: n_action_steps must be <= chunk_size (defaults to chunk_size if not specified)
benchmark_policy = ACT(
    model=ACTModel(
        input_features=benchmark_input_features,
        output_features=benchmark_output_features,
        chunk_size=100,  # Use 100 to match default n_action_steps
        dim_model=256,
        n_encoder_layers=2,
        n_decoder_layers=1,
    )
)
benchmark_policy.to(device)
benchmark_policy.eval()
print(f"✅ Policy ready for benchmarking")

In [ ]:
# Run benchmark evaluation
print("Running benchmark evaluation...")
print("=" * 60)

results = benchmark.evaluate(benchmark_policy)

# Print summary
print(results.summary())

In [ ]:
# Access detailed per-task results
print("\nPer-Task Breakdown:")
print("-" * 60)
for task_result in results.task_results:
    print(f"  {task_result.task_id}:")
    print(f"    Task: {task_result.task_name[:50]}...")
    print(f"    Success Rate: {task_result.success_rate:.1f}%")
    print(f"    Avg Reward: {task_result.avg_reward:.4f}")
    print(f"    Avg Steps: {task_result.avg_episode_length:.1f}")
    print()

### 5.2 Export Results

In [ ]:
import tempfile
from pathlib import Path

# Create temporary directory for outputs
output_dir = Path(tempfile.mkdtemp()) / "results/benchmark"
output_dir.mkdir(parents=True, exist_ok=True)

# Export to JSON (includes metadata, per-task, per-episode data)
json_path = output_dir / "results.json"
results.to_json(json_path)
print(f"✅ Exported to JSON: {json_path}")

# Export to CSV (per-task summary)
csv_path = output_dir / "results.csv"
results.to_csv(csv_path)
print(f"✅ Exported to CSV: {csv_path}")

# View the JSON structure
import json
with open(json_path) as f:
    data = json.load(f)
print(f"\nJSON keys: {list(data.keys())}")
print(f"Metadata: {data['metadata']}")

### 5.3 Multi-Policy Comparison

To compare multiple policies, simply iterate over them with a dict comprehension:

In [ ]:
# Create a second policy (same architecture, different instance)
# In practice, these would be different models (ACT vs Diffusion vs VQ-BeT)
policy_v1 = benchmark_policy
policy_v1.name = "ACT_v1"  # Set a name for identification

policy_v2 = ACT(
    model=ACTModel(
        input_features=benchmark_input_features,
        output_features=benchmark_output_features,
        chunk_size=100,  # Different chunk size
        dim_model=128,   # Smaller model
        n_encoder_layers=1,
        n_decoder_layers=1,
    )
)
policy_v2.name = "ACT_v2_small"
policy_v2.to(device)
policy_v2.eval()

# Compare multiple policies using dict comprehension
print("Comparing multiple policies...")
print("=" * 60)

policies = [policy_v1, policy_v2]
multi_results = {p.name: benchmark.evaluate(p) for p in policies}

# Print comparison
print("\nPolicy Comparison:")
print("-" * 60)
print(f"{'Policy':<20} {'Success Rate':>15} {'Avg Reward':>12}")
print("-" * 60)
for policy_name, result in multi_results.items():
    print(f"{policy_name:<20} {result.overall_success_rate:>14.1f}% {result.mean_reward:>12.4f}")

### 5.4 Video Recording with Benchmark

`LiberoBenchmark` supports built-in video recording for debugging and visualization:

In [ ]:
# Create benchmark with video recording enabled
video_dir = output_dir / "videos"

benchmark_with_video = LiberoBenchmark(
    task_suite="libero_spatial",
    task_ids=[0],           # Single task for demo
    num_episodes=2,
    max_steps=30,
    video_dir=video_dir,    # Enable video recording
    record_mode="failures", # Only save failed episodes ("all", "failures", "successes", "none")
    seed=42,
)

print(f"Video recording enabled to: {video_dir}")
print(f"Record mode: failures (only save failed episodes)")

# Run evaluation with video recording
results_with_video = benchmark_with_video.evaluate(benchmark_policy)

# Check for saved videos
import os
if video_dir.exists():
    videos = list(video_dir.rglob("*.mp4"))
    print(f"\n✅ Videos saved: {len(videos)}")
    for v in videos:
        print(f"   - {v.relative_to(video_dir)}")
else:
    print("\n(No videos saved - all episodes may have succeeded)")

### 5.5 Using Explicit Gyms (Advanced)

For custom configurations, use the base `Benchmark` class with explicit gyms:

In [ ]:
from physicalai.benchmark import Benchmark

# Create gyms with custom configurations
custom_gyms = [
    LiberoGym(
        task_suite="libero_spatial",
        task_id=i,
        observation_height=128,  # Custom resolution
        observation_width=128,
        obs_type="pixels_agent_pos",
    )
    for i in range(2)  # First 2 tasks
]

# Create benchmark with explicit gyms
custom_benchmark = Benchmark(
    gyms=custom_gyms,
    num_episodes=2,
    max_steps=30,
    seed=42,
)

print(f"Custom benchmark: {custom_benchmark}")
print(f"Gym observation size: {custom_gyms[0].observation_height}x{custom_gyms[0].observation_width}")

# Clean up custom gyms
for g in custom_gyms:
    g.close()

## Summary

### What You Learned

**Part 1-2: LiberoGym (Low-Level)**
- `LiberoGym` wraps LIBERO environments with Gymnasium interface
- Observations include images (`image`, `image2`) and state (proprioception)
- Supports multiple task suites (spatial, object, goal, 10, 90)

**Part 3-4: Policy Creation & Evaluation**
- First-party ACT via `ACT(model=ACTModel(...))`
- Automatic feature extraction via `ACT.from_dataset()`
- Third-party LeRobot policies via `LeRobotPolicy` wrapper
- Production evaluation with `rollout()`, `Rollout`, and `evaluate_policy()`

**Part 5: LiberoBenchmark (High-Level)**
- `LiberoBenchmark` for standardized multi-task evaluation
- Automatic gym creation from task suites
- Built-in video recording
- Export to JSON/CSV

### Quick Reference

```python
# Low-level: Single task development
from physicalai.gyms import LiberoGym
gym = LiberoGym(task_suite="libero_10", task_id=0)
result = rollout(gym, policy, max_steps=300)

# High-level: Full benchmark evaluation  
from physicalai.benchmark import LiberoBenchmark
benchmark = LiberoBenchmark(
    task_suite="libero_10",
    num_episodes=20,
    video_dir="./videos",
    record_mode="failures",
)
results = benchmark.evaluate(policy)
results.to_json("results.json")

# Compare multiple policies
all_results = {p.name: benchmark.evaluate(p) for p in [policy1, policy2]}
```

### Task Suites Reference

| Suite | Tasks | Focus | Recommended Episodes |
|-------|-------|-------|---------------------|
| `libero_spatial` | 10 | Spatial reasoning | 50 |
| `libero_object` | 10 | Object generalization | 50 |
| `libero_goal` | 10 | Goal-conditioned | 50 |
| `libero_10` | 10 | Mixed difficulty | 50 |
| `libero_90` | 90 | Comprehensive | 20 |

In [ ]:
# Cleanup
import shutil

# Close benchmark gyms
for g in benchmark.gyms:
    g.close()
for g in benchmark_with_video.gyms:
    g.close()

# Remove temporary directory
shutil.rmtree(output_dir.parent, ignore_errors=True)

print("✅ Cleanup complete!")